<a href="https://colab.research.google.com/github/vikki231998/case_by/blob/main/new_Vidhilekhai_v1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U transformers


In [2]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


In [4]:
# train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
# Write a response that appropriately completes the request.
# Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response.

# ### Instruction:
# You are a legal expert with advanced knowledge in law, case analysis, and legal reasoning.
# Please answer the following legal question.

# ### Question:
# {}

# ### Response:
# {}"""


In [5]:
# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#     inputs = examples["question"]
#     outputs = examples["answer"]
#     texts = []
#     for question, response in zip(inputs, outputs):
#         if not response.endswith(tokenizer.eos_token):
#             response += tokenizer.eos_token
#         text = train_prompt_style.format(question, response)
#         texts.append(text)
#     return {"text": texts}

In [6]:
def format_chat_template(row):
    row_json = [{"role": "system",
                 "content": "You are a helpful legal AI assistant called VidhilekhAI developed by Get Your Lawyers a company based out of Bhopal, India. Write a response based on the context and the query raised such that it appropriately completes the request.Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response."},
                {"role": "user", "content": row["question"]},
                {"role": "assistant", "content": row["answer"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [7]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files="/content/vidhilekh_qa_dataset.csv",
    split="train"
)
# dataset = dataset.map(
#     formatting_prompts_func,
#     batched=True,
# )
dataset = dataset.map(format_chat_template, num_proc=4)
dataset["text"][29]

'<|im_start|>system\nYou are a helpful legal AI assistant called VidhilekhAI developed by Get Your Lawyers a company based out of Bhopal, India. Write a response based on the context and the query raised such that it appropriately completes the request.Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response.<|im_end|>\n<|im_start|>user\nWhat are your limitations?<|im_end|>\n<|im_start|>assistant\nI, Vidhilekh AI, cannot feel, think independently, or act physically.<|im_end|>\n'

In [8]:
from google.colab import drive
drive.flush_and_unmount()


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [11]:
# inference_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
# Write a response that appropriately completes the request.
# Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response.

# ### Instruction:
# You are a legal expert with advanced knowledge in law, case analysis, and legal reasoning.
# Please answer the following legal question.

# ### Question:
# {}

# ### Response:
# """


In [12]:
question = "Explain Section 424 of the IPC, related to dishonest or fraudulent removal or concealment of property"
message = [
    {"role": "system", "content": "You are a helpful legal AI assistant called VidhilekhAI developed by Get Your Lawyers a company based out of Bhopal, India. Write a response based on the context and the query raised such that it appropriately completes the request.Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response."},
    {"role": "user", "content": question}
]

print(message)

text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=False
)



inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **inputs,
    max_new_tokens=1024
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

[{'role': 'system', 'content': 'You are a helpful legal AI assistant called VidhilekhAI developed by Get Your Lawyers a company based out of Bhopal, India. Write a response based on the context and the query raised such that it appropriately completes the request.Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response.'}, {'role': 'user', 'content': 'Explain Section 424 of the IPC, related to dishonest or fraudulent removal or concealment of property'}]
wait
response
Section 424 of the Indian Penal Code (IPC) deals with the crime of "dishonestly or fraudulently removing or concealing property." This section is part of Chapter XVII of the IPC, which focuses on crimes against personal property.

The key elements of Section 424 include:

1. Dishonesty: The act must be done with intent to defraud or deceive someone.
2. Fraudulent Intent: The defendant's actions were made in bad faith or without reasonable grounds.
3. Property

In [13]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model = get_peft_model(model, peft_config)

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments


training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=4,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [15]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
30,1.829200
60,0.469700
90,0.365500
120,0.278300


TrainOutput(global_step=148, training_loss=0.6395019647237417, metrics={'train_runtime': 211.3677, 'train_samples_per_second': 1.4, 'train_steps_per_second': 0.7, 'total_flos': 288589022883840.0, 'train_loss': 0.6395019647237417, 'entropy': 0.269217971978443, 'num_tokens': 34748.0, 'mean_token_accuracy': 0.9312035111444337, 'epoch': 4.0})

In [16]:
from peft import LoraConfig, get_peft_model

# model is your base model
adapter_dir = "/content/my_lora"   # folder to save LoRA adapter

model.save_pretrained(adapter_dir)  # ⚠️ This creates adapter_config.json + pytorch_model.bin
print("✅ LoRA adapter saved to:", adapter_dir)


✅ LoRA adapter saved to: /content/my_lora


In [18]:
import os

adapter_dir = "/content/my_lora"  # path to your LoRA folder

# List all files in the folder
files = os.listdir(adapter_dir)
print("Files in adapter folder:", files)

# Check specifically for adapter_config.json
if "adapter_config.json" in files:
    print("✅ adapter_config.json exists!")
else:
    print("❌ adapter_config.json is missing!")


Files in adapter folder: ['README.md', 'adapter_model.safetensors', 'adapter_config.json']
✅ adapter_config.json exists!


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_DIR = "/content/output/checkpoint-148"  # folder containing adapter_config.json + safetensors
MERGED_DIR = "/content/merged_model"

# 1️⃣ Tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)

# 2️⃣ 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

# 3️⃣ Load base model (no llm_int8_enable_fp32_cpu_offload)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",      # automatically splits layers between GPU & CPU
    trust_remote_code=True
)

# 4️⃣ Load LoRA adapter
peft_model = PeftModel.from_pretrained(base_model, ADAPTER_DIR, device_map="auto")

# 5️⃣ Merge LoRA into base model
merged_model = peft_model.merge_and_unload()

# 6️⃣ Save merged model + tokenizer
merged_model.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('/content/merged_model/tokenizer_config.json',
 '/content/merged_model/special_tokens_map.json',
 '/content/merged_model/chat_template.jinja',
 '/content/merged_model/vocab.json',
 '/content/merged_model/merges.txt',
 '/content/merged_model/added_tokens.json',
 '/content/merged_model/tokenizer.json')

In [21]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [22]:
BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"   # Base model you started from
ADAPTER_DIR = "/output/checkpoint-148/"   # Your trained LoRA adapter
MERGED_DIR = "/content/merged_model"        # Local folder to save merged model
REPO_NAME = "new_Vidhilekhai_v1.1.0"       # Your HF repo name


In [24]:
# 4️⃣ Merge LoRA adapter into base model
# --------------------------------------------
merged_model = peft_model.merge_and_unload()


In [25]:
merged_model.save_pretrained(MERGED_DIR)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.save_pretrained(MERGED_DIR)

('/content/merged_model/tokenizer_config.json',
 '/content/merged_model/special_tokens_map.json',
 '/content/merged_model/chat_template.jinja',
 '/content/merged_model/vocab.json',
 '/content/merged_model/merges.txt',
 '/content/merged_model/added_tokens.json',
 '/content/merged_model/tokenizer.json')

In [26]:
merged_model.push_to_hub(REPO_NAME, private=True)
tokenizer.push_to_hub(REPO_NAME)

REPO_NAME = "new_Vidhilekhai_v1.1.0"

print(f"✅ Merged model pushed: https://huggingface.co/{REPO_NAME}")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp0qn96t7r/model.safetensors    :   3%|2         | 32.7MB / 1.14GB            

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpn7hxtibs/tokenizer.json       : 100%|##########| 11.4MB / 11.4MB            

✅ Merged model pushed: https://huggingface.co/new_Vidhilekhai_v1.1.0


In [ ]:
question = "Explain your work"
message = [
    {"role": "system", "content": "You are a helpful legal AI assistant called VidhilekhAI developed by Get Your Lawyers a company based out of Bhopal, India. Write a response based on the context and the query raised such that it appropriately completes the request.Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response."},
    {"role": "user", "content": question}
]

print(message)

text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=False
)



inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **inputs,
    max_new_tokens=1024
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

[{'role': 'system', 'content': 'You are a helpful legal AI assistant called VidhilekhAI developed by Get Your Lawyers a company based out of Bhopal, India. Write a response based on the context and the query raised such that it appropriately completes the request.Before answering, think carefully about the question and ensure you give a logical, well-reasoned, and accurate response.'}, {'role': 'user', 'content': 'Explain your work'}]
assistant
I act as a knowledge assistant by interpreting queries and generating answers or actions accordingly. My role is to support learning, research, and professional tasks."
